# The demonstration of the tropical precipitation module on the different datasets

###  Loading the necessary packages

In [1]:
from dask_jobqueue import SLURMCluster # pip 
from dask.distributed import Client, progress 
import dask

import re
import matplotlib as mpl
# Define Agg as Backend for matplotlib when no X server is running
mpl.use('Agg')
import socket
import os
import importlib

import inspect
import timeit
import sys

### Function, which reads the status of the user in the slurm queue 

In [ ]:
#!pip install aqua

In [2]:
def squeue_user(username = "$USER"):
    _squeue_user = os.system("squeue --user="+str(username))
    return _squeue_user 

### Storing the path to the current repository into the variable

In [3]:
with os.popen("pwd ") as f:
    _pwd = f.readline()

pwd = re.split(r'[\n]', _pwd)[0]

### Setting the slurm job

In [4]:

extra_args=[
    "--error="+str(pwd)+"/slurm/logs/dask-worker-%j.err",
    "--output="+str(pwd)+"/slurm/output/dask-worker-%j.out"
]

cluster = SLURMCluster(
    name='dask-cluster', 
    cores=256,    
    memory="500 GB", 
    project="bb1153",
    queue= "compute", 
    walltime='04:30:50',
    job_extra=extra_args,
)
client = Client(cluster)
print(cluster.job_script())

/home/b/b382267/mambaforge/envs/tropical-rainfall/lib/python3.10/site-packages/dask_jobqueue/core.py:255: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home/b/b382267/mambaforge/envs/tropical-rainfall/lib/python3.10/site-packages/dask_jobqueue/slurm.py:49: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project will be respected for now, but it will be removed in a future release. If you already set account, project is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)


#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p compute
#SBATCH -A bb1153
#SBATCH -n 1
#SBATCH --cpus-per-task=256
#SBATCH --mem=466G
#SBATCH -t 04:30:50
#SBATCH --error=/work/bb1153/b382267/AQUA/diagnostics/tropical_rainfall/notebooks/slurm/logs/dask-worker-%j.err
#SBATCH --output=/work/bb1153/b382267/AQUA/diagnostics/tropical_rainfall/notebooks/slurm/output/dask-worker-%j.out

/home/b/b382267/mambaforge/envs/tropical-rainfall/bin/python -m distributed.cli.dask_worker tcp://136.172.124.3:40349 --nthreads 16 --nworkers 16 --memory-limit 29.10GiB --name dummy-name --nanny --death-timeout 60



/home/b/b382267/mambaforge/envs/tropical-rainfall/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34173 instead
  warnings.warn(
/home/b/b382267/mambaforge/envs/tropical-rainfall/lib/python3.10/site-packages/dask_jobqueue/core.py:255: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home/b/b382267/mambaforge/envs/tropical-rainfall/lib/python3.10/site-packages/dask_jobqueue/slurm.py:49: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config f

### Loading the slurm job to the queue 

In [5]:
cluster.scale(jobs=1)

### Checking our slurm job in the queue 

In [6]:
squeue_user()

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           4592287   compute dask-wor  b382267 PD       0:00      1 (None)


0

### Canceling the job if it is necessary. Set the job ID manually

In [8]:
Job_ID =   4589766
os.system("scancel " +str(Job_ID)) 

0

### Setting the path to the diagnostic repository 

In [7]:
sys.path.append(str(pwd)+'/../')
import src.shared_func
import src.tr_pr_mod

### Loading the extra function, which can be used in any diagnostic

In [8]:
while True:
    try:
        importlib.reload(src.shared_func)
        from  src.shared_func import time_interpreter,  animation_creator, image_creator,  xarray_attribute_update,  data_size
        break
    except NameError and AttributeError:
        import src.shared_func
        from  src.shared_func import time_interpreter,   animation_creator, image_creator,  xarray_attribute_update,  data_size
        break

### Loading the tropical precipitation module

In [9]:
while True:
    try:
        importlib.reload(src.tr_pr_mod)
        from  src.tr_pr_mod import TR_PR_Diagnostic
        break
    except NameError and AttributeError:
        import src.tr_pr_mod
        from  src.tr_pr_mod import TR_PR_Diagnostic
        break

In [10]:
configdir = '../../../config/'
diagname  = 'tr_pr'
machine   = 'levante'

### Setting the class attributes

In [11]:
diag = TR_PR_Diagnostic()

diag.num_of_bins = 15
diag.first_edge = 0
diag.width_of_bin = 1*10**(-6)/diag.num_of_bins

last_edge = diag.first_edge  + diag.num_of_bins*diag.width_of_bin

### Importing the aqua module 

In [12]:
import aqua

In [13]:
from aqua import Reader
from aqua.reader import catalogue

In [14]:
catalogue(configdir=configdir)

IFS	tco3999-ng5	2.8km experiment, coupled with FESOM
	- ICMGG_atm2d	
	- ICMU_atm2d	
	- ICMU_atm3d	
	- interpolated_global	
	- interpolated_np	
	- interpolated_sp	
	- interpolated_sp_ci	
	- lra-r100-day	
	- lra-r100-mon	
IFS	tco2559-ng5	4km experiment, coupled with FESOM
	- ICMGG_atm2d	
	- ICMU_atm2d	
	- ICMU_atm3d	
	- interpolated_global	
	- interpolated_np	
	- interpolated_sp	
	- lra-r100-day	
	- lra-r100-mon	
IFS	tco1279-orca025	9km baseline, coupled to NEMO, deep conv ON
	- ICMGG_atm2d	
	- ICMU_atm2d	
	- ICMU_atm3d	
	- lra-r100-day	
	- lra-r100-mon	
IFS	test-tco2559	4km experiment, coupled with FESOM
	- ICMGG_atm2d	2d output
	- ICMU_atm3d	3d output

FESOM	tco3999-ng5	2.5km experiment, coupuled with IFS
	- elem_grid	
	- node_grid	
	- np	nearest-neighbor interpolation to lat-lon grid
	- interpolated_global2d	
	- interpolated_global_TS	
	- interpolated_global_UV	
	- interpolated_np	
	- interpolated_sp	
	- original_2d	original 2d output
	- original_3d	original 3d output
FESOM	tco2559-ng

levante:
  args:
    path: ../../../config//levante/catalog.yaml
  description: ''
  driver: intake.catalog.local.YAMLFileCatalog
  metadata: {}


# MSWEP data

In [15]:
reader = Reader(model="MSWEP", exp="past", source="monthly",configdir=configdir)
MSWEP = reader.retrieve()

In [19]:
MSWEP

<xarray.Dataset>
Dimensions:  (lon: 3600, lat: 1800, time: 502)
Coordinates:
  * lon      (lon) float32 -179.9 -179.8 -179.8 -179.6 ... 179.8 179.9 179.9
  * lat      (lat) float32 89.95 89.85 89.75 89.65 ... -89.75 -89.85 -89.95
  * time     (time) datetime64[ns] 1979-02-01 1979-03-01 ... 2020-11-01
Data variables:
    tprate   (time, lat, lon) float32 dask.array<chunksize=(1, 1800, 3600), meta=np.ndarray>
Attributes:
    history:  Created on 2021-02-04 03:412023-04-03 07:16:21 coordinates adju...

### size of entire dataset

In [20]:
data_size(MSWEP)

3252960000

### timestep of dataset is

In [21]:
time_interpreter(MSWEP)

'M'

### Selecting the small part of dataset for tests

In [24]:
MSWEP_small = MSWEP['tprate'][10:20,:,:]

### size of selected chunk

In [26]:
data_size(MSWEP_small)

64800000

### Attribute update

In [27]:
MSWEP_small = xarray_attribute_update(MSWEP_small, MSWEP)

### Load selected chunk into a memory

In [29]:
MSWEP_small = MSWEP_small.compute()

In [28]:
MSWEP_small

<xarray.DataArray 'tprate' (time: 10, lat: 1800, lon: 3600)>
dask.array<getitem, shape=(10, 1800, 3600), dtype=float32, chunksize=(1, 1800, 3600), chunktype=numpy.ndarray>
Coordinates:
  * lon      (lon) float32 -179.9 -179.8 -179.8 -179.6 ... 179.8 179.9 179.9
  * lat      (lat) float32 89.95 89.85 89.75 89.65 ... -89.75 -89.85 -89.95
  * time     (time) datetime64[ns] 1979-12-01 1980-01-01 ... 1980-09-01
Attributes:
    units:         m s**-1
    history:       2023-04-03 07:16:21 variable renamed by AQUA fixer;\n2023-...
    paramId:       172228
    long_name:     Mean total precipitation rate
    cfVarName:     tprate
    shortName:     tprate
    target_units:  m s**-1
    factor:        3.802570537683474e-10
    offset:        0
    src_units:     mm month-1
    units_fixed:   1

### Units of precepitation rate

In [37]:
MSWEP_small.attrs['units']

'm s**-1'

In [31]:
time_interpreter(MSWEP_small) 

'M'

### Name of coordinates

In [32]:
diag.coordinate_names(MSWEP_small)

('lat', 'lon')

### Tropical latitudes

In [34]:
MSWEP_small = diag.ds_per_lat_range(MSWEP_small)

In [35]:
MSWEP_small

<xarray.DataArray 'tprate' (time: 10, lat: 200, lon: 3600)>
array([[[7.46254472e-08, 7.14883228e-08, 7.18685840e-08, ...,
         8.43457642e-08, 8.41318695e-08, 7.46254472e-08],
        [8.03293005e-08, 7.93548907e-08, 7.95450248e-08, ...,
         8.52251105e-08, 8.55103011e-08, 8.03293005e-08],
        [8.24920150e-08, 8.13274781e-08, 8.16602039e-08, ...,
         8.71501626e-08, 8.75304167e-08, 8.24920150e-08],
        ...,
        [1.43451970e-07, 1.52839561e-07, 1.53219830e-07, ...,
         1.44402620e-07, 1.45020536e-07, 1.43451970e-07],
        [1.43285604e-07, 1.52388012e-07, 1.52649434e-07, ...,
         1.44426380e-07, 1.45020536e-07, 1.43285604e-07],
        [1.35609170e-07, 1.44687803e-07, 1.44640268e-07, ...,
         1.39483035e-07, 1.39815768e-07, 1.35680466e-07]],

       [[2.97075822e-08, 2.96362845e-08, 2.97788798e-08, ...,
         2.91847293e-08, 2.89946005e-08, 2.91847293e-08],
        [2.99927763e-08, 3.01353715e-08, 3.05156291e-08, ...,
         2.91847293e-08, 2.93273246e-08, 2.96362845e-08],
        [3.08958867e-08, 3.08008197e-08, 3.11810773e-08, ...,
         2.95174534e-08, 2.99927763e-08, 3.05156291e-08],
...
        [9.60624362e-08, 9.68942473e-08, 9.52543928e-08, ...,
         7.43164890e-08, 7.84280161e-08, 8.12086469e-08],
        [9.22123320e-08, 9.24499943e-08, 9.41373841e-08, ...,
         7.77863320e-08, 8.07808576e-08, 8.04243641e-08],
        [9.19509091e-08, 9.30441431e-08, 9.34244042e-08, ...,
         7.98539830e-08, 8.00678777e-08, 8.21117538e-08]],

       [[1.00815647e-07, 1.01290972e-07, 1.00007604e-07, ...,
         1.06139247e-07, 1.06899762e-07, 1.00649288e-07],
        [1.04974710e-07, 1.05925352e-07, 1.04071603e-07, ...,
         1.07517678e-07, 1.10559739e-07, 1.05117309e-07],
        [1.03691342e-07, 1.04998477e-07, 1.04309258e-07, ...,
         1.07731573e-07, 1.12437256e-07, 1.04309258e-07],
        ...,
        [7.92835948e-08, 8.14463093e-08, 8.02104694e-08, ...,
         7.42689537e-08, 7.52195959e-08, 7.91409960e-08],
        [7.79051632e-08, 8.08521534e-08, 8.13512457e-08, ...,
         7.38173966e-08, 7.46729754e-08, 7.76675009e-08],
        [7.90459325e-08, 7.89033408e-08, 7.88558054e-08, ...,
         7.22488380e-08, 7.37223331e-08, 7.88558054e-08]]], dtype=float32)
Coordinates:
  * lon      (lon) float32 -179.9 -179.8 -179.8 -179.6 ... 179.8 179.9 179.9
  * lat      (lat) float32 9.95 9.85 9.75 9.65 9.55 ... -9.65 -9.75 -9.85 -9.95
  * time     (time) datetime64[ns] 1979-12-01 1980-01-01 ... 1980-09-01
Attributes:
    units:         m s**-1
    history:       2023-04-03 07:16:21 variable renamed by AQUA fixer;\n2023-...
    paramId:       172228
    long_name:     Mean total precipitation rate
    cfVarName:     tprate
    shortName:     tprate
    target_units:  m s**-1
    factor:        3.802570537683474e-10
    offset:        0
    src_units:     mm month-1
    units_fixed:   1

### Fastest histogram

In [43]:
hist_fast_MSWEP  = diag.hist1d_fast(MSWEP_small,  preprocess = False)
hist_fast_MSWEP

<xarray.DataArray (bin: 15)>
array([4.488186e+06, 2.010442e+06, 5.872700e+05, 9.718900e+04,
       1.269400e+04, 3.112000e+03, 7.870000e+02, 1.930000e+02,
       4.800000e+01, 2.300000e+01, 2.500000e+01, 7.000000e+00,
       6.000000e+00, 1.000000e+00, 5.000000e+00])
Coordinates:
  * bin      (bin) float64 0.0 6.667e-08 1.333e-07 ... 8e-07 8.667e-07 9.333e-07
Attributes:
    units:         m s**-1
    history:       2023-04-03 07:16:21 variable renamed by AQUA fixer;\n2023-...
    paramId:       172228
    long_name:     Mean total precipitation rate
    cfVarName:     tprate
    shortName:     tprate
    target_units:  m s**-1
    factor:        3.802570537683474e-10
    offset:        0
    src_units:     mm month-1
    units_fixed:   1

### Plot of obtained histogram 

In [48]:
diag.hist_plot(hist_fast_MSWEP, pdf = True, smooth = True, label='MSWEP', plot_title = "MSWEP, precipitation rate for one timestep")

## Plot of mean and median value of precipitation rate

## Add grid and time units!

In [51]:
diag.mean_and_median_plot(MSWEP_small, savelabel = 'MSWEP')

In [16]:
MSWEP_small = MSWEP['tprate'][10:150,:,:]
MSWEP_small = MSWEP_small.compute()

## Check if vmin and vmax == None. Now its giving an error

In [ ]:
animation_creator(MSWEP_small, vmin =0, vmax = 10**(-7),  trop_lat = 90,  label='MSWEP')

## Add units on bar

In [21]:
image_creator(MSWEP_small, vmin =0, vmax = 10**(-7), trop_lat = 90, label='MSWEP')

Done!
